# 强化学习入门 🎮

## 欢迎来到强化学习的世界！

想象一下，如果你能让计算机学会"玩游戏"和"做决策"，是不是很神奇？这就是强化学习的魅力！

在这个notebook中，我们将一起探索：
- 什么是强化学习？
- 为什么要强化学习？
- 如何用Python实现简单的强化学习？

准备好了吗？让我们开始吧！

## 1. 什么是强化学习？ 🤔

### 生活中的例子
想象你在：
- 学习骑自行车
- 玩电子游戏
- 训练宠物

这就是强化学习！计算机也可以学会这种技能。

### 小测验
1. 下面哪些是强化学习的例子？
   - [x] 训练机器人走路
   - [x] 玩游戏AI
   - [ ] 计算1+1=2
   - [x] 自动驾驶

2. 为什么需要强化学习？
   - 帮助我们解决决策问题
   - 让计算机学会"玩游戏"和"做决策"
   - 提高工作效率

3. 你能想到哪些强化学习的例子？
   - 比如：游戏AI、机器人控制、自动驾驶...

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gym
from gym import spaces

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("工具准备完毕！让我们开始吧！")

## 2. Q学习：让计算机学会做决策 🎯

### 生活中的例子
- 选择最佳路径
- 选择最佳策略
- 选择最佳动作

让我们用Python实现一个简单的Q学习算法！

In [ ]:
# 创建简单的环境
class SimpleEnv:
    def __init__(self):
        self.action_space = spaces.Discrete(2)  # 两个动作：左/右
        self.observation_space = spaces.Discrete(4)  # 四个状态
        self.state = 0
        self.done = False
        
    def reset(self):
        self.state = 0
        self.done = False
        return self.state
    
    def step(self, action):
        if action == 0:  # 向左
            self.state = max(0, self.state - 1)
        else:  # 向右
            self.state = min(3, self.state + 1)
            
        reward = 1 if self.state == 3 else 0
        self.done = self.state == 3
        
        return self.state, reward, self.done, {}

# 创建Q学习算法
class QLearning:
    def __init__(self, states, actions, learning_rate=0.1, discount_factor=0.9):
        self.q_table = np.zeros((states, actions))
        self.lr = learning_rate
        self.gamma = discount_factor
        
    def choose_action(self, state, epsilon=0.1):
        if np.random.random() < epsilon:
            return np.random.randint(2)
        return np.argmax(self.q_table[state])
    
    def learn(self, state, action, reward, next_state):
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state])
        new_value = (1 - self.lr) * old_value + self.lr * (reward + self.gamma * next_max)
        self.q_table[state, action] = new_value

# 训练智能体
env = SimpleEnv()
agent = QLearning(4, 2)
episodes = 1000
rewards = []

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    
    while True:
        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state)
        total_reward += reward
        state = next_state
        
        if done:
            break
            
    rewards.append(total_reward)

# 可视化学习过程
plt.figure(figsize=(10, 4))
plt.plot(rewards)
plt.title('Q学习训练过程')
plt.xlabel('回合数')
plt.ylabel('总奖励')
plt.show()

### 动手做一做
1. 修改环境参数，观察学习效果如何变化
2. 尝试使用不同的学习率
3. 观察Q表的变化

### 思考题
1. 为什么需要Q学习？
2. 你能想到其他可以用Q学习解决的问题吗？

## 3. 策略梯度：让计算机学会优化策略 📈

### 生活中的例子
- 优化游戏策略
- 优化控制策略
- 优化决策策略

让我们学习如何实现策略梯度算法！

In [ ]:
# 创建策略网络
class PolicyNetwork:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        
    def forward(self, state):
        logits = np.dot(state, self.weights)
        probs = np.exp(logits) / np.sum(np.exp(logits))
        return probs
    
    def update(self, states, actions, rewards, learning_rate=0.01):
        for state, action, reward in zip(states, actions, rewards):
            probs = self.forward(state)
            gradient = np.zeros_like(self.weights)
            gradient[:, action] = state
            self.weights += learning_rate * reward * gradient

# 训练策略网络
env = SimpleEnv()
policy = PolicyNetwork(4, 2)
episodes = 1000
rewards = []

for episode in range(episodes):
    state = env.reset()
    states, actions, rewards_episode = [], [], []
    total_reward = 0
    
    while True:
        state_one_hot = np.zeros(4)
        state_one_hot[state] = 1
        probs = policy.forward(state_one_hot)
        action = np.random.choice(2, p=probs)
        
        next_state, reward, done, _ = env.step(action)
        states.append(state_one_hot)
        actions.append(action)
        rewards_episode.append(reward)
        total_reward += reward
        state = next_state
        
        if done:
            break
            
    policy.update(states, actions, rewards_episode)
    rewards.append(total_reward)

# 可视化学习过程
plt.figure(figsize=(10, 4))
plt.plot(rewards)
plt.title('策略梯度训练过程')
plt.xlabel('回合数')
plt.ylabel('总奖励')
plt.show()

### 动手做一做
1. 修改网络结构，观察学习效果如何变化
2. 尝试使用不同的学习率
3. 观察策略的变化

### 思考题
1. 为什么需要策略梯度？
2. 你能想到其他可以用策略梯度解决的问题吗？

## 4. 深度强化学习：让计算机学会更复杂的任务 🧠

### 生活中的例子
- 玩复杂的游戏
- 控制复杂的机器人
- 解决复杂的问题

让我们学习如何实现深度强化学习！

In [ ]:
# 创建深度Q网络
import tensorflow as tf
from tensorflow.keras import layers, models

class DQN:
    def __init__(self, state_size, action_size):
        self.model = models.Sequential([
            layers.Dense(24, activation='relu', input_shape=(state_size,)),
            layers.Dense(24, activation='relu'),
            layers.Dense(action_size, activation='linear')
        ])
        self.model.compile(optimizer='adam', loss='mse')
        
    def choose_action(self, state, epsilon=0.1):
        if np.random.random() < epsilon:
            return np.random.randint(2)
        q_values = self.model.predict(state.reshape(1, -1))
        return np.argmax(q_values[0])
    
    def train(self, states, actions, rewards, next_states, dones):
        targets = self.model.predict(states)
        next_q_values = self.model.predict(next_states)
        
        for i in range(len(states)):
            if dones[i]:
                targets[i][actions[i]] = rewards[i]
            else:
                targets[i][actions[i]] = rewards[i] + 0.9 * np.max(next_q_values[i])
                
        self.model.fit(states, targets, epochs=1, verbose=0)

# 训练深度Q网络
env = SimpleEnv()
dqn = DQN(4, 2)
episodes = 1000
rewards = []

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    
    while True:
        state_one_hot = np.zeros(4)
        state_one_hot[state] = 1
        action = dqn.choose_action(state_one_hot)
        next_state, reward, done, _ = env.step(action)
        next_state_one_hot = np.zeros(4)
        next_state_one_hot[next_state] = 1
        
        dqn.train(
            state_one_hot.reshape(1, -1),
            np.array([action]),
            np.array([reward]),
            next_state_one_hot.reshape(1, -1),
            np.array([done])
        )
        
        total_reward += reward
        state = next_state
        
        if done:
            break
            
    rewards.append(total_reward)

# 可视化学习过程
plt.figure(figsize=(10, 4))
plt.plot(rewards)
plt.title('深度Q学习训练过程')
plt.xlabel('回合数')
plt.ylabel('总奖励')
plt.show()

### 动手做一做
1. 修改网络结构，观察学习效果如何变化
2. 尝试使用不同的优化器
3. 观察训练过程

### 思考题
1. 为什么需要深度强化学习？
2. 你能想到其他可以用深度强化学习解决的问题吗？

## 5. 趣味练习 🎯

### 练习1：简单游戏
1. 设计一个简单的游戏环境
2. 实现一个强化学习算法
3. 训练智能体玩游戏

### 练习2：机器人控制
1. 设计一个简单的机器人环境
2. 实现一个强化学习算法
3. 训练机器人完成任务

### 练习3：决策优化
1. 设计一个简单的决策环境
2. 实现一个强化学习算法
3. 优化决策策略

## 6. 知识总结 📝

### 今天学到了什么？
1. 强化学习的基本概念
2. 不同的强化学习算法
3. 如何用Python实现强化学习

### 下节课预告
下次我们将学习如何将前面学到的知识应用到实际项目中！

## 7. 趣味问答 🤔

1. 选择题
   - 下面哪个不是强化学习的例子？
     a) 训练机器人走路
     b) 计算1+1=2
     c) 玩游戏AI
     d) 自动驾驶

2. 判断题
   - 强化学习就是让计算机"玩游戏"和"做决策"（对/错）
   - 奖励对强化学习很重要（对/错）

3. 开放题
   - 你能想到哪些有趣的强化学习问题？
   - 为什么强化学习很重要？
   - 你觉得强化学习会如何改变我们的生活？